In [1]:
import jupyter_black

jupyter_black.load()

In [2]:
%load_ext autoreload
%autoreload 2

In [3]:
from pyfaidx import Fasta
from pathlib import Path
import re

import uniprot_helper
import ncbi_helper

base = Path("../data")
raw = base / "raw"
file = raw / "BungarusMulticinctus.fasta"
ncbi_dir = base / "ncbi_entries"
uniprot_dir = base / "uniprot_entries"

taxon_mapper = {
    "Pbiv": "Python bivittatus",
    "Cvir": "Crotalus viridis",
    "Dacu": "Deinagkistrodon acutus",
    "Nnaj": "Naja naja",
    "Hcur": "Hydrophis curtus",
    "Bmul": "Bungarus multicinctus",
    "Pgut": "Pantherophis guttatus",
    "Tele": "Thamnophis elegans",
}

In [35]:
fasta = Fasta(file)
a = []


entries = []
uniprot_collector = uniprot_helper.UniProtDataGatherer(uniprot_dir=uniprot_dir)
ncbi_collector = ncbi_helper.NcbiDataGatherer(ncbi_dir=ncbi_dir)
for header, seq in fasta.items():
    ncbi_id = None
    seq = str(seq).replace("-", "")
    species_id = header.split("_")[0]
    if species_id not in taxon_mapper:
        if species_id in ["L", "S", "N", "NX"]:
            # uniprot entries
            m = re.search(r"([A-Z0-9]+)_([A-Z0-9]+_[A-Z]+)$", header).groups()
            acc_id, entry = m
            data = uniprot_collector.get_entry(acc_id=acc_id)
            species, taxon_id = uniprot_collector.parse_taxon(rec=data)
            full_seq, mature_peptide = uniprot_collector.parse_seq(rec=data)
            entry = dict(acc_id=acc_id, species=species, taxon_id=taxon_id)
            if (seq == mature_peptide) or (seq == full_seq):
                entry.update(dict(seq=mature_peptide, full_seq=full_seq))
            else:
                raise Exception(f"Sequence doesn't match its UniProt entry: {acc_id}")
        elif species_id in ["ly6e", "lypd2", "lynx1", "slurp2"]:
            m = re.match(r"^[a-z\d]+_[\w\-]+?[_-]([MNPX]{2}_\d+).*$", header)
            if m is None:
                # impossible to extract/get information (4 entries)
                continue
            ncbi_id = m[1]
        else:
            m = re.search(r"^([A-Z]{,5})_(\w+)(?:\.\w+)?$", header)
            code, ncbi_id = m.groups()
    else:
        # entries that come from genomic DNA
        # TODO: BLASTp
        species = taxon_mapper[species_id]
        entry = dict(species=species, seq=seq)
        # print(header)
        # print(seq)
        # break
        continue
    if ncbi_id is not None:
        ncbi_rec = ncbi_collector.get_record(ncbi_id)
        # TODO: check if record is a UniProt/SwissProt entry: sp crossref
        species, taxon_id = ncbi_collector.parse_taxon(rec=ncbi_rec)
        if species == "Sistrurus catenatus edwardsi":
            species = "Sistrurus catenatus edwardsii"
        # TODO: checker: Counter({'TSA': 356, 'VRT': 203, 'HTC': 16, 'EST': 30})
        div = ncbi_rec["GBSeq_division"]
        full_seq = ncbi_collector.parse_seq(rec=ncbi_rec)
        if full_seq is None:
            continue
        # TODO: create entry
        entry = dict(species=species, taxon_id=taxon_id, full_seq=full_seq)
    entry.update(dict(fasta_id=header, data_origin="paper_zhang"))
    entries.append(entry)

Pbiv_NW_006541926_1_gene_1_mRNA_1
MKAWLLALVVVALVGTDPVDALECCTGFLCLSSKECAEGEEFCISVCKTKGLIVNEVIRRCAKTCTYPEPVNIKCCSTDYCNLFITPHCPLLSLLT


In [ ]:
import pandas as pd
import dset_3FTx

base = Path("../data")
helpers = base / "helpers"
blast_dir = base / "blast_out"
taxon_mapper_file = helpers / "taxon_mapper.json"

df = pd.DataFrame(entries)
df = dset_3FTx.add_taxon_id(df=df, taxon_mapper_file=taxon_mapper_file)
df = dset_3FTx.run_blast(df, blast_dir=blast_dir)

In [8]:
pd.DataFrame(entries)["acc_id"].isna().sum()

435

In [9]:
df.shape

(705, 7)

In [18]:
df["acc_id"].dropna().str.split(",").str.len().value_counts()

1    577
Name: acc_id, dtype: int64

In [19]:
df["db"].value_counts(dropna=False)

NaN     285
TR      236
None    113
SP       71
Name: db, dtype: int64

In [65]:
import idmapping

missing = list(mapper.keys())
for from_db in ["EMBL-GenBank-DDBJ", "EMBL-GenBank-DDBJ_CDS"]:
    # get mapped IDs
    job_id = idmapping.submit_id_mapping(
        from_db=from_db, to_db="UniProtKB", ids=missing
    )
    if idmapping.check_id_mapping_results_ready(job_id):
        link = idmapping.get_id_mapping_results_link(job_id)
        results = idmapping.get_id_mapping_results_search(link)
    # add mapped ID
#     seqdb2gi = {gb: gi for gi, gb in gi2seqdb.items()}
#     for result in results["results"]:
#         gi_nr = seqdb2gi[result["from"]]
#         acc_id = result["to"]["primaryAccession"]
#         df_ritu.loc[df_ritu["gi_number"] == gi_nr, ["acc_id"]] = acc_id
#     # search missing ones in next DB
#     missing = results["failedIds"].copy()
# print(
#     "No UniProt AccID found for"
#     f" {len(results['failedIds'])} sequence(s)"
# )

Fetched: 275 / 275
Fetched: 53 / 53
